# Provably safe ACL and firewall rule changes

Changing ACLs or firewall rules is one of the riskiest update to the network. Even a small error can block connectivity for a large set of critical services or open up sensitive resources to the world at large. 

This notebook shows a 3-step process that uses Batfish to make provably safe and correct changes to ACLs and firewall rules, which we generally call filters. For a broader view of Batfish's support for analyzing filters, check out [this notebook](Analyzing%20ACLs%20and%20Firewall%20Rules.ipynb).
![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/provably-safe-acl-and-firewall-changes?pixel&useReferer) 

## Change scenario

We mimic a scenario where we want to permit HTTP traffic (ports 80 and 8080) from one subnet to another. We will implement this by adding a rule to permit this traffic to our filter, and we will then use Batfish to check if the implementation was correct.

In [1]:
# The traffic to allow
src_prefix = "10.10.10.0/24"
dst_prefix = "18.18.18.0/27"
ip_protocols = ["tcp"]
dst_ports = [80, 8080]

# The node and the name of the filter to change
node_name = "rtr-with-acl"
filter_name = "acl_in"

## Intialize the current (pre-change) snapshot

We start by initializing the pre-change snapshot. In our example, the snapshot contains two devices, and we'll change the ACL on `rtr-with-acl`, whose configuration can be seen [here](https://FILL ME/). 

In [2]:
# Import packages and load questions
%run startup.py

# Initialize a network and snapshot
NETWORK_NAME = "network-example-filters"
CURRENT_SNAPSHOT_NAME = "current"
CURRENT_SNAPSHOT_PATH = "networks/example-filters/current"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(CURRENT_SNAPSHOT_PATH, name=CURRENT_SNAPSHOT_NAME, overwrite=True)

'current'

## Step 1:  Ensure that the intended traffic is not already permitted

Before we make the change to allow the intended traffic, we verify that that traffic is not already permitted---because if it is, we are done and do not need to do anything. We accomplish this using the `reachfilter` question. Given a space of flows, specified using header fields such as source and destination addresses and ports, and a matching condition (e.g., permit, deny) as input, this question finds flows that satisfy the condition. If it reports no flows, that implies a guarantee that no flow within the space satisfies the condition. 

In [3]:
# Check if the intended traffic is already permitted in the initial snapshot
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         ipProtocols=ip_protocols,
                         filters=filter_name,
                         nodes = node_name,
                         query="permit").answer(snapshot=CURRENT_SNAPSHOT_NAME).frame()
display_html(answer)

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace


Since the query above did not find any result, we know with certainty that no flow within the specified space is already permitted. We can now proceed. If some flow is returned as part of the query, we may want to first delete the corresponding line. 

## Load candidate change

Assume that we implemented the change as shown by the diff below and stored at `CANDIDATE1_SNAPSHOT_PATH` (see below). 

```
diff -r networks/example-filters/current/configs/rtr-with-acl.cfg \ 
        networks/example-filters/candidate1/configs/rtr-with-acl.cfg
39a40,41
>   462 permit tcp 10.10.10.0/24 18.18.18.0/26 eq 80   
>   463 permit tcp 10.10.10.0/24 18.18.18.0/26 eq 8080   
```

Now we will load the proposed change into Batfish so that we can analyze it. 

In [4]:
CANDIDATE1_SNAPSHOT_NAME = "candidate1"
CANDIDATE1_SNAPSHOT_PATH = "networks/example-filters/candidate1"

bf_init_snapshot(CANDIDATE1_SNAPSHOT_PATH, name=CANDIDATE1_SNAPSHOT_NAME, overwrite=True)

'candidate1'

## Step 2: Ensure that the intended traffic is now permitted

We run the same query as in Step 2 above, but on the candidate1 snapshot.

In [5]:
# Check if any flow in the intended traffic is denied in candidate1 snapshot
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         ipProtocols=ip_protocols,
                         filters=filter_name,
                         nodes=node_name,
                         query="deny").answer(snapshot=CANDIDATE1_SNAPSHOT_NAME).frame()
display_html(answer)

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace


Since we got no results, we can be confident that our candidate change permits *all* traffic that we intended to permit. If there were any flow in the desired space that was not permitted by the change, the query above would have found it.

## Step 3: Ensure that no collateral damage has occurred

Typically, engineers will stop change validation after checking that the intended traffic has been successfully permitted by the change. However, for safety and correctness, we must also check that no traffic outside of the intended space has been impacted--that is, our change has not caused collateral damage. 

We can verify that by using a "differential" version of the `reachfilter` question that compares two snapshots. The query below compares the candidate1 and initial snapshots, and is asking Batfish if there is *any* flow outside of the intended traffic that the two snapshots treat differently (i.e., one of them permits and the other rejects, or vice versa). The "outside" part is specified using the `complementHeaderSpace` flag. If this query, returns no result, then combined with the result above, we have ensured that the change is completely correct.

In [6]:
# Check if traffic other than the intended traffic has been impacted
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         ipProtocols=ip_protocols,
                         filters=filter_name,
                         nodes=node_name,
                         complementHeaderSpace=True).answer(snapshot=CANDIDATE1_SNAPSHOT_NAME, 
                                                            reference_snapshot=CURRENT_SNAPSHOT_NAME).frame()
display_html(answer)

,Node,Filter_Name,Flow,KeyPresence,Diff_Action,Base_Action,Delta_Action,Diff_Line_Number,Base_Line_Number,Delta_Line_Number,Diff_Line_Content,Base_Line_Content,Delta_Line_Content,Diff_Trace,Base_Trace,Delta_Trace
0,rtr-with-acl,acl_in,rtr-with-acl->[10.10.10.0:0->18.18.18.32:80 proto: TCP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,In both,,PERMIT,DENY,: -78,23,101,,462 permit tcp 10.10.10.0/24 18.18.18.0/26 eq 80,2020 deny tcp any any,,"Flow permitted by 'extended ipv4 access-list' named 'acl_in', index 23: 462 permit tcp 10.10.10.0/24 18.18.18.0/26 eq 80","Flow denied by 'extended ipv4 access-list' named 'acl_in', index 101: 2020 deny tcp any any"


As we can see, unfortunately, we do get some results. In particular, we have accidentally allowed traffic that we didn't intend to. The column `Flow` shows the example flow that the two snapshots treat differently. The columns that start with `Base_` show how the candidate snapshot treats that flow, and those that start with `Delta_` show how the current snapshot treats the flow. As show, the candidate snapshot permits the flow while the current snapshot denies it. That means we've accidentally opened up more space than we intended. 

The root cause of the problem is apparent if we look at the diff above more carefully. We ended permitting 18.18.18.0/26 destination prefix as opposed to 18.18.18.0/27. We thus need to fix this.

## Load (another) candidate change

Assume that we implemented another candidate change as shown by the diff below and stored at `CANDIDATE2_SNAPSHOT_PATH` (see below). 

```
diff -r networks/example-filters/current/configs/rtr-with-acl.cfg \ 
        networks/example-filters/candidate2/configs/rtr-with-acl.cfg
39a40,41
>   462 permit tcp 10.10.10.0/24 18.18.18.0/27 eq 80   
>   463 permit tcp 10.10.10.0/24 18.18.18.0/27 eq 8080   
```

We will now load this change and repeat Steps 2 and 3 to validate it.

In [7]:
CANDIDATE2_SNAPSHOT_NAME = "candidate2"
CANDIDATE2_SNAPSHOT_PATH = "networks/example-filters/candidate2"

bf_init_snapshot(CANDIDATE2_SNAPSHOT_PATH, name=CANDIDATE2_SNAPSHOT_NAME, overwrite=True)

'candidate2'

## Step 2 (again): Ensure that the intended traffic is now permitted

We run the same query as in Step 2 above, but on the candidate2 snapshot.

In [8]:
# Check if any flow in the intended traffic is denied in candidate2 snapshot
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         ipProtocols=ip_protocols,
                         filters=filter_name,
                         nodes=node_name,
                         query="deny").answer(snapshot=CANDIDATE2_SNAPSHOT_NAME).frame()
display_html(answer)

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace


As before, we got no results, which means that no flow in the intended traffic is being denied, and we thus correctly permitted all intended traffic. 

## Step 3 (again): Ensure that no collateral damage has occurred

Now, lets also check again that no other traffic is impacted.

In [9]:
# Check if traffic other than the intended traffic has been impacted
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         ipProtocols=ip_protocols,
                         filters=filter_name,
                         nodes=node_name,
                         complementHeaderSpace=True).answer(snapshot=CANDIDATE2_SNAPSHOT_NAME, 
                                                            reference_snapshot=CURRENT_SNAPSHOT_NAME).frame()
display_html(answer)

,Node,Filter_Name,Flow,KeyPresence,Diff_Action,Base_Action,Delta_Action,Diff_Line_Number,Base_Line_Number,Delta_Line_Number,Diff_Line_Content,Base_Line_Content,Delta_Line_Content,Diff_Trace,Base_Trace,Delta_Trace


We got no results again. Thus, this change is completely correct: It allows all traffic that we meant to allow and has no impact of any other traffic. 

## Conclusions

In this notebook, we showed how you can use Batfish to validate that a change to a complex ACL/filter is correct and only permits or denies the desired set of traffic/flows. The following 3 steps provide that guarantee:

1. Check that the intended traffic does not already match the desired action (permit or deny) in the existing configuration
2. Check that the intended traffic is matched with the desired action in the changed configuration
3. Check that nothing but the intended traffic is impacted by the change

This simple 3 step process gives you the assurance needed to be able to quickly and safely make changes to ACLs and firewall rules in your network. For general filter analysis using Batfish, see [this notebook](Analyzing%20ACLs%20and%20Firewall%20Rules.ipynb).

***
### Get involved with the Batfish community

To get involved and learn more, join our community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). 